In [1]:
import os
import numpy as np, h5py, tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd

BASE_DIR = "/Volumes/gurindapalli/projects/trial_classification/4tone_cell"
OUT_DIR  = "/Volumes/gurindapalli/projects/trial_classification/Jason"
os.makedirs(OUT_DIR, exist_ok=True)

# ===== match better version defaults =====
DS_FACTOR = 8          # downsample by 8 like your better script
EPOCHS    = 15
BATCH_SIZE= 32
N_SPLITS  = 5
VAL_SPLIT = 0.1
n_classes = 4

In [2]:
def build_model(input_len, n_classes=4, l2=1e-5, sdrop=0.10, head_drop=0.30):
    inp = layers.Input(shape=(input_len,1))
    x = layers.Conv1D(128, 9, padding="same", activation="relu")(inp)
    x = layers.SpatialDropout1D(sdrop)(x)
    x = layers.MaxPooling1D(2)(x)

    x = layers.Conv1D(128, 9, padding="same", activation="relu")(x)
    x = layers.SpatialDropout1D(sdrop)(x)
    x = layers.MaxPooling1D(2)(x)

    x = layers.Conv1D(256, 9, padding="same", activation="relu")(x)
    x = layers.SpatialDropout1D(sdrop)(x)

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(head_drop)(x)

    out = layers.Dense(
        n_classes, activation="softmax",
        kernel_regularizer=tf.keras.regularizers.l2(l2)
    )(x)

    model = models.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [3]:
def subaverage(X, y, n_classes=4, group_size=5, repeats=6, bootstrap=True, renorm=True):
    """
    Build pseudo-trials by averaging 'group_size' trials of the same class.
    Returns (X_new, y_new) or (None, None) if none created.
    Use ONLY on TRAIN (not across splits).
    """
    rng = np.random.default_rng()
    idx_by_c = [np.where(y == c)[0] for c in range(n_classes)]
    if any(len(idx) < group_size for idx in idx_by_c):   # if any class too small, just skip SA
        return None, None
    groups_per = min(len(idx)//group_size for idx in idx_by_c) * repeats
    if groups_per <= 0:
        return None, None

    X_new, y_new = [], []
    for c, idx in enumerate(idx_by_c):
        for _ in range(groups_per):
            take = rng.choice(idx, size=group_size, replace=bootstrap)
            x_avg = X[take].mean(axis=0)  # (T,1) if X has channel
            if renorm:
                m = x_avg.mean(axis=0, keepdims=True)
                s = x_avg.std(axis=0, keepdims=True) + 1e-7
                x_avg = (x_avg - m) / s
            X_new.append(x_avg); y_new.append(c)
    return np.stack(X_new, axis=0), np.array(y_new, dtype=np.int32)

In [4]:
def decode_labels_like_better(f):
    lab = f["labels"]                                     # cell array of refs (1 x N)
    raw = np.array([int(np.array(f[lab[0, i]])[0, 0])     # deref each cell to scalar
                    for i in range(lab.shape[1])])
    u = np.sort(np.unique(raw))
    tone_to_idx = {v:i for i, v in enumerate(u)}          # per-file rank remap → 0..3
    y = np.array([tone_to_idx[v] for v in raw], dtype=np.int32)
    return y

In [5]:
def load_like_better(mat_path, dataset="ffr_dss", ds_factor=8):
    with h5py.File(mat_path, "r") as f:
        X = f[dataset][()]                 # expected (N, 4997); transpose if needed
        if X.shape[0] == 4997 and X.ndim == 2:
            X = X.T
        y = decode_labels_like_better(f)

    # downsample like better script
    if ds_factor is not None and ds_factor > 1:
        X = X[:, ::ds_factor]

    # per-trial z-norm across time, then add channel dim
    m = X.mean(axis=1, keepdims=True); s = X.std(axis=1, keepdims=True) + 1e-7
    X = ((X - m) / s)[..., np.newaxis]
    return X, y

In [6]:
def run_cv_on_file(MAT_PATH):
    X, y = load_like_better(MAT_PATH, dataset="ffr_dss", ds_factor=DS_FACTOR)
    print(f"{os.path.basename(MAT_PATH)} -> X={X.shape}, y={y.shape}, uniques={np.unique(y)}")
    print("class counts:", np.bincount(y, minlength=n_classes).tolist())

    counts = np.bincount(y, minlength=n_classes)
    if (counts < N_SPLITS).any():
        raise ValueError(f"{os.path.basename(MAT_PATH)}: some class has < {N_SPLITS} samples: {counts.tolist()}")

    skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True)  # same as better version
    fold_accs = []
    cm_total  = np.zeros((n_classes, n_classes), dtype=int)

    for i, (tr, te) in enumerate(skf.split(X, y), 1):
        # explicit stratified train/val from training fold (same as better version)
        sss = StratifiedShuffleSplit(n_splits=1, test_size=VAL_SPLIT)
        tr_sub_idx, val_idx = next(sss.split(X[tr], y[tr]))
        Xtr, ytr = X[tr][tr_sub_idx], y[tr][tr_sub_idx]
        Xval, yval = X[tr][val_idx],   y[tr][val_idx]
        Xte,  yte  = X[te],            y[te]

        # Train-only sub-averages (same as better version)
        Xtr_sa, ytr_sa = subaverage(Xtr, ytr, n_classes, group_size=5, repeats=6, bootstrap=True, renorm=True)
        if Xtr_sa is not None:
            Xtr_aug = np.concatenate([Xtr, Xtr_sa], axis=0)
            ytr_aug = np.concatenate([ytr, ytr_sa], axis=0)
        else:
            Xtr_aug, ytr_aug = Xtr, ytr

        # Build model (same as better version)
        model = build_model(X.shape[1], n_classes)

        # Bias-init to priors of augmented train set (same as better version)
        priors = np.bincount(ytr_aug, minlength=n_classes) / len(ytr_aug)
        last = model.layers[-1]
        if hasattr(last, "bias") and last.bias is not None:
            last.bias.assign(np.log(priors + 1e-8))

        # Simple class-weights from augmented counts (same as better version)
        class_counts = np.bincount(ytr_aug, minlength=n_classes).astype(np.float32)
        class_weight = {k: (len(ytr_aug) / (n_classes * max(float(c), 1.0))) for k, c in enumerate(class_counts)}

        es  = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
        rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

        print(f"\n{os.path.basename(MAT_PATH)} | Fold {i}: counts  "
              f"train={np.bincount(ytr, minlength=n_classes).tolist()}  "
              f"val={np.bincount(yval, minlength=n_classes).tolist()}  "
              f"test={np.bincount(yte,  minlength=n_classes).tolist()}  "
              f"priors={np.round(priors,3).tolist()}")

        model.fit(
            Xtr_aug, ytr_aug,
            validation_data=(Xval, yval),   # val is single-trial (no SA), same as better version
            epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0,
            callbacks=[es, rlr],
            class_weight=class_weight
        )

        # Evaluate on held-out test fold (same as better version)
        y_hat = np.argmax(model.predict(Xte, batch_size=BATCH_SIZE, verbose=0), axis=1)
        acc   = accuracy_score(yte, y_hat)
        fold_accs.append(acc)
        cm_total += confusion_matrix(yte, y_hat, labels=list(range(n_classes)))

        # Debug: prediction usage per fold
        pred_counts = np.bincount(y_hat, minlength=n_classes)
        print(f"{os.path.basename(MAT_PATH)} | Fold {i}: acc={acc:.4f} | pred counts={pred_counts.tolist()}")

    print(f"{os.path.basename(MAT_PATH)} | Mean acc: {np.mean(fold_accs):.4f} ± {np.std(fold_accs):.4f}")
    return fold_accs, cm_total

In [7]:
file_ids = list(range(1002, 1016))  # 1002..1015 inclusive
accs_by_file = {}                   # { "4T1002": [fold accs], ... }
labels = []
agg_rows = []

for fid in file_ids:
    fname = f"4T{fid}.mat"
    mat_path = os.path.join(BASE_DIR, fname)
    if not os.path.exists(mat_path):
        print(f"Missing file: {mat_path} (skipping)")
        continue

    fold_accs, cm_total = run_cv_on_file(mat_path)
    key = f"4T{fid}"
    accs_by_file[key] = fold_accs
    labels.append(key)

    row = {"file": key}
    for i in range(n_classes):
        for j in range(n_classes):
            row[f"cm_{i}{j}"] = int(cm_total[i, j])
    agg_rows.append(row)

    # free up memory between runs
    tf.keras.backend.clear_session()

agg_cols = ["file"] + [f"cm_{i}{j}" for i in range(n_classes) for j in range(n_classes)]
df_agg = pd.DataFrame(agg_rows, columns=agg_cols)
agg_csv_path = os.path.join(OUT_DIR, "cm_all_files.csv")
df_agg.to_csv(agg_csv_path, index=False)
print(f"\nSaved aggregated confusion matrices → {agg_csv_path}")

print("\nPer-file mean ± SD across CV folds:")
for k in labels:
    m = float(np.mean(accs_by_file[k]))
    sd = float(np.std(accs_by_file[k]))
    print(f"{k}: {m:.4f} ± {sd:.4f}")

PermissionError: [Errno 13] Unable to synchronously open file (unable to lock file, errno = 13, error message = 'Permission denied')